In [3]:
from flask import Flask, render_template_string, request, send_from_directory
import networkx as nx
import matplotlib.pyplot as plt
import heapq
import os
import threading

app = Flask(__name__)

# --- Define the graph ---
graph = {
    'A': [('B', 4), ('H', 8)],
    'B': [('A', 4), ('C', 8), ('H', 11)],
    'C': [('B', 8), ('D', 7), ('F', 4), ('I', 2)],
    'D': [('C', 7), ('E', 9), ('F', 14)],
    'E': [('D', 9), ('F', 10)],
    'F': [('C', 4), ('D', 14), ('E', 10), ('G', 2)],
    'G': [('F', 2), ('H', 1), ('I', 6)],
    'H': [('A', 8), ('B', 11), ('G', 1), ('I', 7)],
    'I': [('C', 2), ('G', 6), ('H', 7)]
}

# --- Prim's Algorithm ---
def prims_algorithm(graph, start):
    mst = []
    visited = set()
    min_heap = [(0, start, None)]
    while min_heap:
        weight, current, from_node = heapq.heappop(min_heap)
        if current in visited:
            continue
        visited.add(current)
        if from_node:
            mst.append((from_node, current, weight))
        for neighbor, edge_weight in graph[current]:
            if neighbor not in visited:
                heapq.heappush(min_heap, (edge_weight, neighbor, current))
    return mst

# --- Dijkstra's Algorithm ---
def dijkstra(graph, start, end):
    heap = [(0, start, [])]
    visited = set()
    while heap:
        (cost, current, path) = heapq.heappop(heap)
        if current in visited:
            continue
        visited.add(current)
        path = path + [current]
        if current == end:
            return (cost, path)
        for neighbor, weight in graph[current]:
            if neighbor not in visited:
                heapq.heappush(heap, (cost + weight, neighbor, path))
    return (float("inf"), [])

# --- Draw MST Graph ---
def draw_graph(mst):
    G = nx.Graph()
    for u in graph:
        for v, w in graph[u]:
            G.add_edge(u, v, weight=w)
    pos = nx.spring_layout(G, seed=42)
    mst_edges = [(u, v) for u, v, w in mst]
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, edge_color='lightgray')
    nx.draw_networkx_edges(G, pos, edgelist=mst_edges, edge_color='red', width=2)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title("Air Traffic Control - Minimum Spanning Tree")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("static/mst_graph.png")
    plt.close()

# --- Draw Dijkstra Path ---
def draw_shortest_path(path):
    G = nx.Graph()
    for u in graph:
        for v, w in graph[u]:
            G.add_edge(u, v, weight=w)
    pos = nx.spring_layout(G, seed=42)
    path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1000)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='green', width=3)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title("Shortest Path between Airports")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("static/shortest_path.png")
    plt.close()

# --- HTML Template ---
HTML_TEMPLATE = """
<!doctype html>
<html>
<head>
    <title>Air Traffic Control</title>
    <style>
        body { font-family: Arial; background: #eef; padding: 20px; }
        h1, h2 { color: #2c3e50; text-align: center; }
        table { width: 60%; border-collapse: collapse; margin: 20px auto; }
        th, td { padding: 10px; border: 1px solid #aaa; text-align: center; }
        img { margin: 20px auto; display: block; max-width: 90%; border: 1px solid #ccc; }
        form { text-align: center; margin-top: 30px; }
        select, button { padding: 8px 12px; margin: 10px; border-radius: 6px; border: 1px solid #888; }
        button { background-color: #3498db; color: white; cursor: pointer; transition: 0.3s; }
        button:hover { background-color: #2980b9; }
        .highlight { color: green; font-weight: bold; }
    </style>
</head>
<body>
    <h1>Air Traffic Control System</h1>
    <h2>Minimum Spanning Tree (MST)</h2>
    <table>
        <tr><th>From</th><th>To</th><th>Distance</th></tr>
        {% for u, v, w in mst %}
        <tr><td>{{ u }}</td><td>{{ v }}</td><td>{{ w }}</td></tr>
        {% endfor %}
    </table>
    <h3 class="highlight" style="text-align:center">Total Distance: {{ total }}</h3>
    <img src="/static/mst_graph.png" alt="MST Graph">

    <h2>Find Shortest Path Between Airports</h2>
    <form method="post" action="/shortest">
        <label>From:</label>
        <select name="source">
            {% for node in graph %}<option value="{{ node }}">{{ node }}</option>{% endfor %}
        </select>
        <label>To:</label>
        <select name="destination">
            {% for node in graph %}<option value="{{ node }}">{{ node }}</option>{% endfor %}
        </select>
        <button type="submit">Find Shortest Path</button>
    </form>

    {% if path %}
        <h3 class="highlight" style="text-align:center">Shortest Path: {{ path|join(' → ') }}</h3>
        <h3 class="highlight" style="text-align:center">Total Distance: {{ cost }}</h3>
        <img src="/static/shortest_path.png" alt="Shortest Path">
    {% endif %}
</body>
</html>
"""

# --- Flask Routes ---
@app.route('/')
def index():
    mst = prims_algorithm(graph, 'A')
    total = sum(w for u, v, w in mst)
    draw_graph(mst)
    return render_template_string(HTML_TEMPLATE, mst=mst, total=total, path=None, cost=None, graph=graph)

@app.route('/shortest', methods=['POST'])
def shortest_path():
    source = request.form['source']
    destination = request.form['destination']
    cost, path = dijkstra(graph, source, destination)
    draw_shortest_path(path)
    mst = prims_algorithm(graph, 'A')
    total = sum(w for u, v, w in mst)
    draw_graph(mst)
    return render_template_string(HTML_TEMPLATE, mst=mst, total=total, path=path, cost=cost, graph=graph)

@app.route('/static/<filename>')
def serve_image(filename):
    return send_from_directory('static', filename)

# --- Start Flask in background thread ---
def run_flask():
    app.run(debug=False, use_reloader=False)

if __name__ == "__main__":
    if not os.path.exists("static"):
        os.makedirs("static")
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
